##3030 Assignment 7 kNN Model Solution

In [2]:
# Example of kNN implemented from Scratch in Python

import csv
import random
import math
import operator

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'rb') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        
        # Convert observations to float and capture max and min value of each predictor
        # Initialize minval and maxval to first observation
        minval = []
        maxval = []
        for y in range(4):
            minval.append(float(dataset[0][y]))
            maxval.append(float(dataset[0][y]))
               
        for x in range(len(dataset)): # note there appears to be a bug in original code that would drop the last obseration
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
                minval[y] = min(minval[y], dataset[x][y])
                maxval[y] = max(maxval[y], dataset[x][y])
        
        # Normalize all predictors to between 0 and 1
        # Alternatively, if we have reason to believe the data is Normally distributed we could normalize to units of Z
        # by finding the mean and std. dev. of each predictor, then adjusting each of the predictor observations by subtracting
        # the mean and dividing by the s.d. (Z = (x - mu) / sigma)
        # Here though we're subtracting the min value and dividing by the range
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = (dataset[x][y] - minval[y]) / (maxval[y] - minval[y])
        print dataset
        
        # Split dataset into training and test
        for x in range(len(dataset)):
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])


def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

# Add a function for Manhattan distance
def manhattanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += abs(instanc1[x] - instance2[x])
    return distance

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        # to use Manhattan distance substitute above with
        # dist = manhattanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

# To use for numeric prediction:
# We need the response (i.e. the target) to be a value rather than a class label, but assuming that it is, alter the above
# as follows and use this function instead
# Rather than return the label with the most votes, this returns the average of the k nearest neighbors.
def getPrediction(neighbors):
    est = 0.0
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        est += response
    return est/len(neighbors)

def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

def main():
    # prepare data
    trainingSet=[]
    testSet=[]
    split = 0.67
    loadDataset('iris.dat', split, trainingSet, testSet)
    print 'Train set: ' + repr(len(trainingSet))
    print 'Test set: ' + repr(len(testSet))
    # generate predictions
    predictions=[]
    k = 3
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')

main()

[[0.22222222222222213, 0.6249999999999999, 0.06779661016949151, 0.04166666666666667, 'Iris-setosa'], [0.1666666666666668, 0.41666666666666663, 0.06779661016949151, 0.04166666666666667, 'Iris-setosa'], [0.11111111111111119, 0.5, 0.05084745762711865, 0.04166666666666667, 'Iris-setosa'], [0.08333333333333327, 0.4583333333333333, 0.0847457627118644, 0.04166666666666667, 'Iris-setosa'], [0.19444444444444448, 0.6666666666666666, 0.06779661016949151, 0.04166666666666667, 'Iris-setosa'], [0.30555555555555564, 0.7916666666666665, 0.11864406779661016, 0.12500000000000003, 'Iris-setosa'], [0.08333333333333327, 0.5833333333333333, 0.06779661016949151, 0.08333333333333333, 'Iris-setosa'], [0.19444444444444448, 0.5833333333333333, 0.0847457627118644, 0.04166666666666667, 'Iris-setosa'], [0.027777777777777922, 0.3749999999999999, 0.06779661016949151, 0.04166666666666667, 'Iris-setosa'], [0.1666666666666668, 0.4583333333333333, 0.0847457627118644, 0.0, 'Iris-setosa'], [0.30555555555555564, 0.708333333